In [1]:
from nuplan.common.maps.nuplan_map.map_factory import get_maps_api, get_maps_db

MAP_LOCATIONS = {"sg-one-north", "us-ma-boston", "us-nv-las-vegas-strip", "us-pa-pittsburgh-hazelwood"}

In [2]:
import geopandas as gpd
import pyogrio


# Path to your .gpkg file
# path_to_gpkg = "/home/daniel/nuplan/dataset/maps/us-nv-las-vegas-strip/9.15.1915/map.gpkg"
path_to_gpkg = "/home/daniel/nuplan/dataset/maps/us-ma-boston/9.12.1817/map.gpkg"
# path_to_gpkg = "/home/daniel/nuplan/dataset/maps/us-pa-pittsburgh-hazelwood/9.17.1937/map.gpkg"
# path_to_gpkg = "/home/daniel/nuplan/dataset/maps/sg-one-north/9.17.1964/map.gpkg"


# List all available layers
layers = [
    "baseline_paths",
    "carpark_areas",
    "generic_drivable_areas",
    "dubins_nodes",
    "lane_connectors",
    "intersections",
    "boundaries",
    "crosswalks",
    "lanes_polygons",
    "lane_group_connectors",
    "lane_groups_polygons",
    "walkways",
    "gen_lane_connectors_scaled_width_polygons",
    "meta",
]


def load_layer(layer_name: str) -> gpd.geodataframe:


    map_meta = gpd.read_file(path_to_gpkg, layer="meta", engine="pyogrio")
    projection_system = map_meta[map_meta["key"] == "projectedCoordSystem"]["value"].iloc[0]

    gdf_in_pixel_coords = pyogrio.read_dataframe(path_to_gpkg, layer=layer_name, fid_as_index=True)
    gdf_in_utm_coords = gdf_in_pixel_coords.to_crs(projection_system)

    # For backwards compatibility, cast the index to string datatype.
    #   and mirror it to the "fid" column.
    gdf_in_utm_coords.index = gdf_in_utm_coords.index.map(str)
    gdf_in_utm_coords["fid"] = gdf_in_utm_coords.index

    return gdf_in_utm_coords

pyogrio.read_dataframe(path_to_gpkg)

/home/daniel/anaconda3/envs/asim/lib/python3.12/site-packages/pyogrio/geopandas.py:275: UserWarning: More than one layer found in 'map.gpkg': 'baseline_paths' (default), 'carpark_areas', 'generic_drivable_areas', 'dubins_nodes', 'lane_connectors', 'intersections', 'boundaries', 'crosswalks', 'lanes_polygons', 'lane_group_connectors', 'lane_groups_polygons', 'road_segments', 'stop_polygons', 'traffic_lights', 'walkways', 'gen_lane_connectors_scaled_width_polygons', 'meta'. Specify layer parameter to avoid this warning.
  result = read_func(


,path_type_fid,creator_id,lane_fid,lane_connector_fid,dubins_node_fids,geometry
0,1,0015b69d-0796-4348-b834-b3e20b2097d6,47592.0,NaN,"35021,33755","LINESTRING (-71.0455 42.34899, -71.0455 42.348..."
1,1,0045a0d1-8bb2-4e18-be2a-6de339497e24,51256.0,NaN,"34478,33444","LINESTRING (-71.04841 42.34101, -71.04841 42.3..."
2,1,00586662-cbac-4fde-a8d3-ce919742c65b,NaN,53099.0,"34031,33147","LINESTRING (-71.04083 42.34096, -71.04082 42.3..."
3,1,00889254-7ec2-45aa-a484-34e05322b8c8,NaN,52635.0,"32290,32305","LINESTRING (-71.04914 42.35234, -71.04914 42.3..."
4,1,00a31c4d-866b-47d6-bd47-0beba5e1a83a,NaN,52204.0,"32705,33380","LINESTRING (-71.02999 42.34651, -71.02998 42.3..."
...,...,...,...,...,...,...
3014,0,6efe16d8-8f1a-4505-b6a8-684d3755a991,51103.0,NaN,"34190,34327","LINESTRING (-71.03257 42.34743, -71.03257 42.3..."
3015,0,c521b2b8-5bbb-4bf7-844b-e9deded2e4d4,NaN,52394.0,"34327,33740","LINESTRING (-71.03188 42.34714, -71.03188 42.3..."
3016,0,688c47e6-a4b1-42e5-ac4d-64fa94855100,NaN,51880.0,"34327,34434","LINESTRING (-71.03188 42.34714, -71.03188 42.3..."
3017,0,63be1df6-f2c1-4c92-ad76-c20e1152a0f5,NaN,52553.0,"33770,32846","LINESTRING (-71.03155 42.34705, -71.03155 42.3..."


In [12]:
[
    "carpark_areas",
    "generic_drivable_areas",
    "dubins_nodes",
    "lane_connectors",
    "intersections",
    "boundaries",
    "crosswalks",
    "lanes_polygons",
    "lane_group_connectors",
    "lane_groups_polygons",
    "road_segments",
    "stop_polygons",
    "traffic_lights",
    "walkways",
    "gen_lane_connectors_scaled_width_polygons",
    "meta",
]

lanes = load_layer("lanes_polygons")
boundaries = load_layer("boundaries")


def get_lane(id: int):
    return lanes[lanes["fid"] == str(id)]

def get_right_boundary(lane):
    return boundaries[boundaries["fid"] == str(lane["right_boundary_fid"].iloc[0])]

def get_left_boundary(lane):
    return boundaries[boundaries["fid"] ==  str(lane["left_boundary_fid"].iloc[0])]


# boundaries

# correct_lane_fid = 47716
# error_lane_fid = 48508

# correct_lane_df = get_lane(correct_lane_fid)
# error_lane_df = get_lane(error_lane_fid)



# 46552,46553
# test_id = 46553

lanes

,lane_group_fid,lane_fid,lane_index,creator_id,name,road_type_fid,lane_type_fid,width,left_offset,right_offset,...,stops,left_boundary_fid,right_boundary_fid,speed_limit_mps,left_has_reflectors,right_has_reflectors,from_edge_fid,to_edge_fid,geometry,fid
fid,,,,,,,,,,,,,,,,,,,,,
46832,48806,46832,1,f430f151-31c4-41c8-aca8-bb8dd19f49af,None,1,0,12,NaN,NaN,...,NaN,62767,62769,NaN,False,False,56728,56729,"POLYGON ((331646.62 4690274.598, 331647.607 46...",46832
46838,49396,46838,1,7074e7ac-f7cb-412b-ab33-4e92cbedfe92,None,1,0,12,NaN,NaN,...,NaN,62771,62773,NaN,False,False,56730,56731,"POLYGON ((330954.391 4689791.394, 330948.363 4...",46838
46859,49007,46859,3,579bd12f-3b80-4998-9206-0b250ea15444,None,1,0,12,NaN,NaN,...,NaN,45771,62775,NaN,False,False,56732,56733,"POLYGON ((331404.649 4690614.997, 331414.834 4...",46859
46860,49007,46860,1,e5ca641b-75b6-4678-a38d-4c901a4b1c7c,None,1,0,12,NaN,NaN,...,NaN,62777,46302,NaN,False,False,56734,56735,"POLYGON ((331444.995 4690595.065, 331437.869 4...",46860
46861,49007,46861,2,e621171b-f633-458b-be03-cb1e223b9ca7,None,1,0,12,NaN,NaN,...,NaN,46302,45771,NaN,False,False,56736,56737,"POLYGON ((331402.992 4690612.3, 331410.612 469...",46861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51857,48809,51857,1,fe5918c5-a30e-43ef-b31f-a71f2e986a74,None,1,0,12,NaN,NaN,...,NaN,46596,46204,NaN,False,False,59178,59179,"POLYGON ((331174.4 4690801.717, 331178.932 469...",51857
51858,49030,51858,1,fed1588e-c136-4d36-bf82-4169383c73d1,None,1,0,12,NaN,NaN,...,NaN,46724,65173,NaN,False,False,59180,59181,"POLYGON ((331180.387 4690555.575, 331171.001 4...",51858
51859,49477,51859,1,ff41f22e-7a44-45d6-9ed6-c671c5a73118,None,1,0,12,NaN,NaN,...,NaN,65175,65177,NaN,False,False,59182,59183,"POLYGON ((331014.487 4691173.604, 331019.564 4...",51859


In [11]:
lanes[lanes["right_boundary_fid"] == test_id]

,creator_id,left_boundary_fid,right_boundary_fid,from_edge_fid,to_edge_fid,geometry,fid
fid,,,,,,,


In [ ]:
boundaries[boundaries["fid"] == str(test_id)]

In [6]:
def _get_set(df, column: str):
    """
    Returns a set of unique values in the specified column of the DataFrame.
    """
    return set(df[column].unique())

In [ ]:
_get_set(lanes, "lane_index")

In [ ]:
lanes[lanes["left_boundary_fid"] == test_id].keys()

In [ ]:
get_left_boundary(correct_lane_df)

In [ ]:
get_right_boundary(correct_lane_df)

In [ ]:
get_left_boundary(error_lane_df)

In [ ]:
get_right_boundary(error_lane_df)

In [ ]:

import pandas as pd
lane_df =  load_layer("generic_drivable_areas")
lane_df

In [ ]:
walkways_df = load_layer("carparks")
walkways_df

In [ ]:

gen_lane_connectors_scaled_width_polygons_df = load_layer("boundaries")

gen_lane_connectors_scaled_width_polygons_df[gen_lane_connectors_scaled_width_polygons_df["fid"] == "17950"]

In [ ]:
[None] * 10

In [ ]:
import pandas as pd
lane_df =  load_layer("baseline_paths")
lane_df

In [ ]:
def mps_to_mph(mph: float) -> float:
    """
    Convert meters per second to miles per hour.
    :param mph: miles per hour [mi/h]
    :return: meters per second [m/s]
    """
    return mph / 0.44704
mps_to_mph(6.705409029950827)

In [ ]:
# fix, ax = plt.subplots(figsize=(10, 10))

# type = 3

# for i in np.random.choice(len(geoms[type]), 10):
#     ax.plot(*geoms[type][i].coords.xy, color="blue")


In [ ]:
geoms[2][1]

In [ ]:
geoms[3][1]